In [1]:
!gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS
To: /kaggle/working/hw7_data.zip
100%|██████████████████████████████████████| 12.1M/12.1M [00:00<00:00, 48.7MB/s]


In [2]:
!unzip -o hw7_data.zip

Archive:  hw7_data.zip
  inflating: hw7_train.json          
  inflating: hw7_test.json           
  inflating: hw7_dev.json            
  inflating: hw7_in-context-learning-examples.json  


In [5]:
# 先卸载冲突版本
!pip uninstall transformers accelerate tokenizers -y

# 安装兼容版本
!pip install transformers==4.41.0 accelerate==0.21.0 sentence-transformers kaggle-environments

Found existing installation: transformers 4.26.1
Uninstalling transformers-4.26.1:
  Successfully uninstalled transformers-4.26.1
Found existing installation: accelerate 0.16.0
Uninstalling accelerate-0.16.0:
  Successfully uninstalled accelerate-0.16.0
Found existing installation: tokenizers 0.13.3
Uninstalling tokenizers-0.13.3:
  Successfully uninstalled tokenizers-0.13.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 69.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 96.0 MB/s eta 0:00:00:00:01


In [7]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
	torch.manual_seed(seed)
	if torch.cuda.is_available():
			torch.cuda.manual_seed(seed)
			torch.cuda.manual_seed_all(seed)
	np.random.seed(seed)
	random.seed(seed)
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True
same_seeds(2)

print("import complete!")

import complete!


In [15]:
from transformers import (
  AutoTokenizer,
  AutoModelForQuestionAnswering,
)

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-chinese").to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

print("import complete!")

# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

import complete!


In [16]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

train_questions, train_paragraphs = read_data("hw7_train.json")
dev_questions, dev_paragraphs = read_data("hw7_dev.json")
test_questions, test_paragraphs = read_data("hw7_test.json")

print("function defined!")

function defined!


In [18]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False)

print("complete!")

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

complete!


In [35]:
import random

class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 60
        self.max_paragraph_len = 150
        self.min_paragraph_len = 50 # 新增最小长度
        self.doc_stride = 75 # 减小步长
        

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### Preprocessing这块准备大改 #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            
            # 初始化默认值
            answer_start_token = -1
            answer_end_token = -1
            paragraph_start = 0
            
            # 10%概率使用无答案样本,减弱计算机幻觉
            if random.random() < 0.1:
                answer_start_token = answer_end_token = -1
                paragraph_start = random.randint(0,max(0,len(tokenized_paragraph) - self.max_paragraph_len))

            else:
                answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
                answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])

                # 随即偏移窗口，防止模型思维定式,只知道从中间开始找
                offset = random.randint(-self.max_paragraph_len // 4, self.max_paragraph_len // 4)
                mid = (answer_start_token + answer_end_token) // 2 + offset
                paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, 
                                     len(tokenized_paragraph) - self.max_paragraph_len))
                
            # 动态长度
            current_paragraph_len = random.randint(self.min_paragraph_len,self.max_paragraph_len)
            paragraph_end = min(paragraph_start + current_paragraph_len, len(tokenized_paragraph))
            

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		

            if answer_start_token != -1:
                answer_start_token += len(input_ids_question) - paragraph_start
                answer_end_token += len(input_ids_question) - paragraph_start
            else:
                answer_start_token = answer_end_token = 0 # 无答案时设置为0
                
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

print("class defined!")

class defined!


In [36]:
def evaluate(data, output, tokenizer):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ""
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1] # 获取窗口数量
    
    for k in range(num_of_windows):
        # 获取当前窗口的start和end logits
        start_logits = output.start_logits[k]
        end_logits = output.end_logits[k]
        
        # 找到概率最高的start和end位置
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)

        # 检查是否为无效答案（start > end 或 指向[CLS]/[SEP]）
        if start_index > end_index:
            continue # 跳过无效窗口
        # 计算联合概率（可改用对数概率或其他加权方式）
        prob = start_prob + end_prob
        
       # 更新最佳答案
        if prob > max_prob:
            max_prob = prob
            # 解码token（跳过特殊符号如[CLS]、[SEP]）
            tokens = data[0][0][k][start_index : end_index + 1]
            answer = tokenizer.decode(tokens, skip_special_tokens = True)

    # 后处理: 去除空格,无效符号等
    answer = answer.replace(' ','' )
    
     # 如果所有窗口均无效，返回空字符串
    return answer if answer else ""

print("function defined!s")

function defined!s


In [39]:
from accelerate import Accelerator
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

# hyperparameters
num_epoch = 3
validation = True
logging_step = 3000
learning_rate = 3e-5
train_batch_size = 4 
gradient_accumulation_steps = 16
warmup_ratio = 0.1  # 学习率warmup比例
fp16_training = True
# 初始化Accelerator
accelerator = Accelerator(
    mixed_precision="fp16" if fp16_training else "no",
    gradient_accumulation_steps=gradient_accumulation_steps
)

# 数据加载器
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

# 优化器和学习率调度
optimizer = AdamW(model.parameters(), lr=learning_rate, correct_bias=False)
total_steps = len(train_loader) * num_epoch
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(total_steps * warmup_ratio),
    num_training_steps=total_steps
)

model, optimizer, train_loader, scheduler = accelerator.prepare(
    model, optimizer, train_loader, scheduler
) 


#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can make train_batch_size lower and gradient_accumulation_steps upper
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation


# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/

print("Start Training ...")

for epoch in range(num_epoch):
    model.train()
    total_loss = 0
    correct_predictions = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}")
    for step, data in enumerate(progress_bar):
        data = [i.to(accelerator.device) for i in data]

        with accelerator.accumulate(model):
            output = model(
                input_ids=data[0],
                token_type_ids=data[1],
                attention_mask=data[2],
                start_positions=data[3],
                end_positions=data[4]
            )
            loss = output.loss
            accelerator.backward(loss)
            
            # 梯度裁剪
            if accelerator.sync_gradients:
                accelerator.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            
            # 计算准确率
            start_pred = torch.argmax(output.start_logits, dim=1)
            end_pred = torch.argmax(output.end_logits, dim=1)
            correct = ((start_pred == data[3]) & (end_pred == data[4])).float().mean()
            
            total_loss += loss.item()
            correct_predictions += correct.item()
            
            # 更新进度条
            progress_bar.set_postfix({
                'loss': total_loss / (step + 1),
                'acc': correct_predictions / (step + 1),
                'lr': scheduler.get_last_lr()[0]
            })
        
        # 定期日志记录
        if (step + 1) % logging_step == 0:
            avg_loss = total_loss / logging_step
            avg_acc = correct_predictions / logging_step
            print(f"\nEpoch {epoch+1} | Step {step+1} | Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f}")
            total_loss = correct_predictions = 0

    # 验证集评估
    if validation:
        print("\nEvaluating Dev Set...")
        model.eval()
        dev_correct = 0
        
        with torch.no_grad():
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(
                    input_ids=data[0].squeeze(0).to(accelerator.device),
                    token_type_ids=data[1].squeeze(0).to(accelerator.device),
                    attention_mask=data[2].squeeze(0).to(accelerator.device)
                )
                pred_answer = evaluate(data, output, tokenizer)
                dev_correct += int(pred_answer == dev_questions[i]["answer_text"])
        
        dev_acc = dev_correct / len(dev_loader)
        print(f"Validation | Epoch {epoch+1} | Acc: {dev_acc:.4f}")

# 保存模型
print("\nSaving Model...")
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained("saved_model")

Start Training ...


Epoch 1:  45%|████▍     | 3002/6730 [02:49<03:23, 18.32it/s, loss=0.00351, acc=0.00025, lr=2.78e-6] 


Epoch 1 | Step 3000 | Loss: 4.1369 | Acc: 0.0648


Epoch 1:  89%|████████▉ | 6002/6730 [05:38<00:41, 17.35it/s, loss=0.00106, acc=0.00025, lr=5.56e-6] 


Epoch 1 | Step 6000 | Loss: 2.5821 | Acc: 0.3207


...


Epoch 3 | Step 6000 | Loss: 1.7540 | Acc: 0.5306


Epoch 3: 100%|██████████| 6730/6730 [06:18<00:00, 17.76it/s, loss=0.194, acc=0.0572, lr=1.88e-5]    



Evaluating Dev Set...


100%|██████████| 2863/2863 [01:03<00:00, 44.95it/s]


Validation | Epoch 3 | Acc: 0.7461

Saving Model...


In [42]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output, tokenizer))

result_file = "result.csv"
with open(result_file, 'w') as f:	
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


100%|██████████| 3524/3524 [01:21<00:00, 43.47it/s]

Completed! Result is in result.csv
